In [1]:
sc

In [4]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## Saving and Loading Models

In [5]:
from pyspark.ml.linalg import Vectors

df = sqlc.createDataFrame([(1.0, Vectors.dense(1.0, 2.0, 3.0)),
                           (1.0, Vectors.dense(2.0, 3.0, 4.0)),
                           (0.0, Vectors.dense(-1.0, 1.0, 2.0)),
                           (0.0, Vectors.dense(-2.0, 3.0, 5.0))]).toDF("label", "features")

In [7]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()

lrModel = lr.fit(df)
lrModel.transform(df).show()

+-----+--------------+--------------------+--------------------+----------+
|label|      features|       rawPrediction|         probability|prediction|
+-----+--------------+--------------------+--------------------+----------+
|  1.0| [1.0,2.0,3.0]|[-18.070405604445...|[1.41945802370848...|       1.0|
|  1.0| [2.0,3.0,4.0]|[-38.987081234651...|[1.16983808020729...|       1.0|
|  0.0|[-1.0,1.0,2.0]|[19.2085506510254...|[0.99999999545187...|       0.0|
|  0.0|[-2.0,3.0,5.0]|[29.1902958840818...|[0.99999999999978...|       0.0|
+-----+--------------+--------------------+--------------------+----------+



In [8]:
lrModel.save("lrModel.parquet")

In [10]:
from pyspark.ml.classification import LogisticRegressionModel

sameModel = LogisticRegressionModel.load("lrModel.parquet")
sameModel.transform(df).show()

+-----+--------------+--------------------+--------------------+----------+
|label|      features|       rawPrediction|         probability|prediction|
+-----+--------------+--------------------+--------------------+----------+
|  1.0| [1.0,2.0,3.0]|[-18.070405604445...|[1.41945802370848...|       1.0|
|  1.0| [2.0,3.0,4.0]|[-38.987081234651...|[1.16983808020729...|       1.0|
|  0.0|[-1.0,1.0,2.0]|[19.2085506510254...|[0.99999999545187...|       0.0|
|  0.0|[-2.0,3.0,5.0]|[29.1902958840818...|[0.99999999999978...|       0.0|
+-----+--------------+--------------------+--------------------+----------+



In [13]:
!cat lrModel.parquet/metadata/part-00000 

{"class":"org.apache.spark.ml.classification.LogisticRegressionModel","timestamp":1475089043445,"sparkVersion":"2.0.0","uid":"LogisticRegression_43d69252fb4e6810e320","paramMap":{"regParam":0.0,"tol":1.0E-6,"fitIntercept":true,"maxIter":100,"standardization":true,"elasticNetParam":0.0,"probabilityCol":"probability","rawPredictionCol":"rawPrediction","featuresCol":"features","labelCol":"label","predictionCol":"prediction","threshold":0.5}}
